In [75]:
# Import necessary libraries 
import sklearn
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial.distance import cosine
from scipy.spatial.distance import jaccard
import scipy
import timeit
%pylab inline
pylab.rcParams['figure.figsize'] = (10, 6)

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [76]:
# Ingest data
df = pd.read_csv("C:\\GIT\\Santander\\Felix\\One_month_data.csv")
df.head()

,ncodpers,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,1043896,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1046978,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1040902,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1032046,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1080030,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [77]:
# Check for null values and drop ID column
df.isnull().any()
df1 = df.drop('ncodpers', 1)

In [78]:
df1.head()

,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [79]:
# Create placeholder dataframe listing item vs item
data_ibs = pd.DataFrame(index=df1.columns, columns=df1.columns)

In [80]:
data_ibs.head()

,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
ind_ahor_fin_ult1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ind_aval_fin_ult1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ind_cco_fin_ult1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ind_cder_fin_ult1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ind_cno_fin_ult1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
# Fill in those empty spaces with jaccard similarities (cosine can also be used but is not as accurate)
# Loop through the columns
for i in range(0,len(data_ibs.columns)) :
    # Loop through the columns for each column
    for j in range(0,len(data_ibs.columns)) :
    # Fill in placeholder with jaccard similarities
      data_ibs.ix[i,j] = 1-jaccard(df1.ix[1:,i],df1.ix[1:,j])

In [82]:
# Create a placeholder items for closes neighbours to an item
data_neighbours = pd.DataFrame(index=data_ibs.columns,columns=range(1, 11))
 # Loop through our similarity dataframe and fill in neighbouring item names
for i in range(0,len(data_ibs.columns)):
    data_neighbours.ix[i,:10] = data_ibs.ix[0:,i].sort_values(ascending=False)[:10].index
    
# End Item Based Recommendations

In [83]:
data_neighbours

,1,2,3,4,5,6,7,8,9,10
ind_ahor_fin_ult1,ind_recibo_ult1,ind_nom_pens_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1
ind_aval_fin_ult1,ind_recibo_ult1,ind_nom_pens_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1
ind_cco_fin_ult1,ind_cco_fin_ult1,ind_ctop_fin_ult1,ind_recibo_ult1,ind_ecue_fin_ult1,ind_dela_fin_ult1,ind_ctpp_fin_ult1,ind_valo_fin_ult1,ind_ctma_fin_ult1,ind_reca_fin_ult1,ind_deco_fin_ult1
ind_cder_fin_ult1,ind_recibo_ult1,ind_nom_pens_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1
ind_cno_fin_ult1,ind_cno_fin_ult1,ind_nom_pens_ult1,ind_nomina_ult1,ind_recibo_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_ecue_fin_ult1,ind_ctpp_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1
ind_ctju_fin_ult1,ind_ctju_fin_ult1,ind_recibo_ult1,ind_nom_pens_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1
ind_ctma_fin_ult1,ind_ctma_fin_ult1,ind_cno_fin_ult1,ind_nomina_ult1,ind_recibo_ult1,ind_nom_pens_ult1,ind_cco_fin_ult1,ind_tjcr_fin_ult1,ind_ecue_fin_ult1,ind_reca_fin_ult1,ind_deco_fin_ult1
ind_ctop_fin_ult1,ind_ctop_fin_ult1,ind_cco_fin_ult1,ind_dela_fin_ult1,ind_reca_fin_ult1,ind_ecue_fin_ult1,ind_valo_fin_ult1,ind_recibo_ult1,ind_tjcr_fin_ult1,ind_ctpp_fin_ult1,ind_cno_fin_ult1
ind_ctpp_fin_ult1,ind_ctpp_fin_ult1,ind_reca_fin_ult1,ind_cno_fin_ult1,ind_nom_pens_ult1,ind_tjcr_fin_ult1,ind_recibo_ult1,ind_nomina_ult1,ind_dela_fin_ult1,ind_cco_fin_ult1,ind_ctop_fin_ult1
ind_deco_fin_ult1,ind_deco_fin_ult1,ind_valo_fin_ult1,ind_cco_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ecue_fin_ult1,ind_recibo_ult1,ind_deme_fin_ult1,ind_aval_fin_ult1,ind_cder_fin_ult1


In [84]:
# --- Start User Based Recommendations --- #
 
# Helper function to get similarity scores
def getScore(history, similarities):
    return sum(history*similarities)/sum(similarities)

In [85]:
# Create a place holder matrix for similarities, and fill in the user name column
data_sims = pd.DataFrame(index=df.index,columns=df.columns)
data_sims.ix[:,:1] = df.ix[:,:1]

In [90]:
#Loop through all rows, skip the user column, and fill with similarity scores
tic = timeit.default_timer()
for i in range(0,len(data_sims.index)):
    for j in range(1,len(data_sims.columns)):
        user = data_sims.index[i]
        product = data_sims.columns[j]
 
        if df.ix[i][j] == 1:
            data_sims.ix[i][j] = 0
        else:
            product_top_names = data_neighbours.ix[product][1:10]
            product_top_sims = data_ibs.ix[product].sort_values(ascending=False)[1:10]
            user_purchases = df1.ix[user,product_top_names]
 
            data_sims.ix[i][j] = getScore(user_purchases,product_top_sims)
toc = timeit.default_timer()
print('Load Time',toc - tic)

('Load Time', 41.798061041085305)


In [45]:
# Get the top products
data_recommend = pd.DataFrame(index=data_sims.index, columns=['ncodpers','1','2','3','4','5','6','7'])
data_recommend.ix[0:,0] = data_sims.ix[:,0]

In [91]:
# Instead of top product scores, we want to see names
tic=timeit.default_timer()
for i in range(0,len(data_sims.index)):
    data_recommend.ix[i,1:] = data_sims.ix[i,:].sort_values(ascending=False).ix[1:8,].index.transpose()
toc=timeit.default_timer()
print('Load Time',toc - tic)

('Load Time', 0.7674522822801464)


In [47]:
# Print a sample
data_recommend.head(7)

,ncodpers,1,2,3,4,5,6,7
0,1043896,ind_pres_fin_ult1,ind_ctop_fin_ult1,ind_deco_fin_ult1,ind_dela_fin_ult1,ind_ctma_fin_ult1,ind_hip_fin_ult1,ind_recibo_ult1
1,1046978,ind_pres_fin_ult1,ind_ctop_fin_ult1,ind_deco_fin_ult1,ind_dela_fin_ult1,ind_ctma_fin_ult1,ind_hip_fin_ult1,ind_recibo_ult1
2,1040902,ind_pres_fin_ult1,ind_ctop_fin_ult1,ind_deco_fin_ult1,ind_dela_fin_ult1,ind_recibo_ult1,ind_ctma_fin_ult1,ind_tjcr_fin_ult1
3,1032046,ind_pres_fin_ult1,ind_ctop_fin_ult1,ind_deco_fin_ult1,ind_dela_fin_ult1,ind_ctma_fin_ult1,ind_hip_fin_ult1,ind_recibo_ult1
4,1080030,ind_dela_fin_ult1,ind_cco_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1
5,921941,ind_pres_fin_ult1,ind_ctop_fin_ult1,ind_deco_fin_ult1,ind_dela_fin_ult1,ind_ctma_fin_ult1,ind_hip_fin_ult1,ind_recibo_ult1
6,970089,ind_pres_fin_ult1,ind_ctop_fin_ult1,ind_deco_fin_ult1,ind_dela_fin_ult1,ind_ctma_fin_ult1,ind_hip_fin_ult1,ind_recibo_ult1


In [48]:
# Change results to correct format for Kaggle submission
test_preds = []

for i in data_recommend.drop('ncodpers', axis=1).values:
    test_preds.append(' '.join(i))
    
df = data_recommend.drop(['1', '2', '3', '4', '5', '6', '7'], axis=1)
df['added_products'] = test_preds
df2 = df.drop_duplicates()

In [49]:
df2.head()

,ncodpers,added_products
0,1043896,ind_pres_fin_ult1 ind_ctop_fin_ult1 ind_deco_f...
1,1046978,ind_pres_fin_ult1 ind_ctop_fin_ult1 ind_deco_f...
2,1040902,ind_pres_fin_ult1 ind_ctop_fin_ult1 ind_deco_f...
3,1032046,ind_pres_fin_ult1 ind_ctop_fin_ult1 ind_deco_f...
4,1080030,ind_dela_fin_ult1 ind_cco_fin_ult1 ind_cno_fin...


In [50]:
# Save recommendations 
df2.to_csv('C:\GIT\Santander\Felix\Example_Engine.csv', index=False) 

In [73]:
# Gives accuracy of model using Root Mean Squared Error (smaller number = better recommendations)
from math import sqrt
rms = sqrt(mean_squared_error(user_purchases,product_top_sims))
rms

0.10080621030727492